<a href="https://colab.research.google.com/github/FahmidaNourin/CSE445_Machine_Learning/blob/main/Project_Initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#connecting with drive

In [136]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#read in csv file

In [137]:
import numpy as np
import pandas as pd


df = pd.read_csv("/content/drive/MyDrive/Sleep_Research/HMC_Sleep_Stages.csv")

In [138]:
df.head()

,Subject,Sleep_Stage,Epoch,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,SpectralEdge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,SpectralEdge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,SpectralEdge_Gamma_O2,PeakF_Gamma_O2
0,SN002,N1,6,0.00034,10.11512,17.85756,17.48293,10.24000,0.00047,17.73268,...,0.00112,0.74927,6.36878,2.87220,0.62439,0.000076,32.71805,43.83220,49.95122,49.95122
1,SN002,N1,7,0.00035,10.73951,17.60780,17.10829,9.11610,0.00048,16.85854,...,0.00070,0.99902,6.86829,3.62146,0.74927,0.000072,32.09366,44.33171,49.95122,49.95122
2,SN002,N1,8,0.00035,9.36585,17.60780,16.98341,9.24098,0.00042,17.35805,...,0.00077,1.49854,6.49366,4.12098,0.74927,0.000069,33.09268,43.95707,49.95122,49.95122
3,SN002,N1,116,0.00054,9.74049,15.98439,15.48488,8.36683,0.00051,14.61073,...,0.00082,0.87415,7.49268,3.49659,0.87415,0.000039,28.34732,43.08293,49.95122,49.95122
4,SN002,N1,117,0.00034,10.11512,16.48390,15.98439,12.23805,0.00034,15.11024,...,0.00084,0.87415,6.86829,2.87220,0.62439,0.000032,28.97171,43.20780,49.95122,49.95122


In [139]:
df = df.dropna().reset_index(drop=True)

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108451 entries, 0 to 108450
Data columns (total 78 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Subject                108451 non-null  object 
 1   Sleep_Stage            108451 non-null  object 
 2   Epoch                  108451 non-null  int64  
 3   MeanP_Alpha_F4         108451 non-null  float64
 4   MedianF_Alpha_F4       108451 non-null  float64
 5   MeanF_Alpha_F4         108451 non-null  float64
 6   SpectralEdge_Alpha_F4  108451 non-null  float64
 7   PeakF_Alpha_F4         108451 non-null  float64
 8   MeanP_Beta_F4          108451 non-null  float64
 9   MedianF_Beta_F4        108451 non-null  float64
 10  MeanF_Beta_F4          108451 non-null  float64
 11  SpectralEdge_Beta_F4   108451 non-null  float64
 12  PeakF_Beta_F4          108451 non-null  float64
 13  MeanP_Theta_F4         108451 non-null  float64
 14  MedianF_Theta_F4       108451 non-nu

#Converting N1,N2 -> Light_Sleep, N3 -> Deep_Sleep And removing W stage.

In [141]:
df.shape

(108451, 78)

In [142]:
df =df.replace("N1", "Light_Sleep")
df =df.replace("N2", "Light_Sleep")
df =df.replace("N3", "Deep_Sleep")
df =df.replace("R", "REM")
df.drop(df.index[df['Sleep_Stage'] == "W"], inplace = True)


In [143]:
df.shape

(89096, 78)

#Removing columns

In [144]:
df = df.loc[:,df.columns != "Subject"]
df = df.loc[:,df.columns != "Epoch"]


#Encoding target columns (Sleep Stage)

In [145]:
target="Sleep_Stage"

In [146]:
df[target].value_counts()

Light_Sleep    51341
Deep_Sleep     21290
REM            16465
Name: Sleep_Stage, dtype: int64

In [147]:
from sklearn.preprocessing import LabelEncoder
encd = LabelEncoder()

In [148]:
df[target] = encd.fit_transform(df[target])

In [149]:
df[target].value_counts()

1    51341
0    21290
2    16465
Name: Sleep_Stage, dtype: int64

#Feature Selection: ANOVA

In [150]:

from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd

X = df.loc[:,df.columns != "Sleep_Stage"]
y = df["Sleep_Stage"]

fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA']

lyst = featureScores.nlargest(15,'Score_ANOVA')
ANOVA_feature=list(lyst['Best_columns'])
ANOVA_feature


['MeanF_Alpha_F4',
 'SpectralEdge_Alpha_F4',
 'MedianF_Alpha_O2',
 'MeanF_Delta_C4',
 'MeanF_Alpha_C4',
 'MeanF_Delta_F4',
 'MedianF_Alpha_F4',
 'SpectralEdge_Alpha_C4',
 'MeanF_Delta_O2',
 'SpectralEdge_Alpha_O2',
 'SpectralEdge_Beta_F4',
 'MeanF_Alpha_O2',
 'SpectralEdge_Beta_C4',
 'SpectralEdge_Delta_C4',
 'MedianF_Beta_F4']

#Pearson’s

In [151]:

from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd
X = df.loc[:,df.columns != "Sleep_Stage"]
y = df["Sleep_Stage"]
fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

#create df for scores
dfscores = pd.DataFrame(fit.scores_)
#create df for column names
dfcolumns = pd.DataFrame(df.columns)
#concat two dataframes for better visualization
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#naming the dataframe columns
featureScores.columns = ['Best_columns','Score_pearsons']

#print 10 best features
lyst = featureScores.nlargest(15,'Score_pearsons')

PEARSON_feature=list(lyst['Best_columns'])
PEARSON_feature


['MedianF_Alpha_F4',
 'MeanF_Alpha_F4',
 'MedianF_Alpha_C4',
 'MeanP_Alpha_O2',
 'MeanF_Alpha_C4',
 'MedianF_Delta_C4',
 'MedianF_Delta_F4',
 'MedianF_Delta_O2',
 'MeanF_Alpha_O2',
 'MedianF_Alpha_O2',
 'MeanF_Beta_F4',
 'SpectralEdge_Beta_F4',
 'MeanF_Beta_C4',
 'MeanP_Beta_F4',
 'MeanF_Delta_C4']

#Spliting Dataset into X(features) and y(target)

In [152]:
from scipy.stats import pearson3
#X = df.loc[:,df.columns != target]
#X = df.loc[:,ANOVA_feature]
X = df.loc[:,PEARSON_feature]
y = df[target]




#Scaling Dataset

In [153]:
X

,MedianF_Alpha_F4,MeanF_Alpha_F4,MedianF_Alpha_C4,MeanP_Alpha_O2,MeanF_Alpha_C4,MedianF_Delta_C4,MedianF_Delta_F4,MedianF_Delta_O2,MeanF_Alpha_O2,MedianF_Alpha_O2,MeanF_Beta_F4,SpectralEdge_Beta_F4,MeanF_Beta_C4,MeanP_Beta_F4,MeanF_Delta_C4
0,10.11512,17.85756,9.86537,0.00023,18.48195,0.87415,0.74927,0.74927,18.48195,10.61463,25.35024,27.47317,25.84976,0.00047,6.74341
1,10.73951,17.60780,10.48976,0.00023,19.60585,0.87415,0.74927,0.99902,18.10732,10.36488,25.22537,26.97366,25.84976,0.00048,6.61854
2,9.36585,17.60780,8.99122,0.00025,17.35805,1.62341,0.99902,1.49854,17.35805,9.74049,26.09951,28.47220,26.34927,0.00042,6.36878
3,9.74049,15.98439,10.24000,0.00029,17.10829,0.87415,0.74927,0.87415,16.73366,9.49073,21.72878,22.47805,22.22829,0.00051,7.61756
4,10.11512,16.48390,9.86537,0.00020,17.10829,0.74927,0.62439,0.87415,17.10829,9.99024,22.10341,23.47707,22.85268,0.00034,6.49366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89091,9.36585,17.60780,9.61561,0.00023,17.23317,1.37366,0.99902,0.99902,17.60780,9.36585,22.72780,24.10146,23.10244,0.00026,6.24390
89092,10.36488,20.10537,10.61463,0.00044,20.35512,0.99902,0.49951,0.49951,20.10537,10.36488,30.22049,35.34049,30.47024,0.00083,6.61854
89093,10.24000,18.48195,10.73951,0.00046,18.73171,0.99902,0.74927,0.74927,18.48195,10.24000,27.59805,30.34537,27.59805,0.00066,6.99317
89094,9.24098,16.48390,9.24098,0.00032,16.23415,1.49854,0.62439,0.62439,16.48390,9.24098,23.97659,24.97561,22.60293,0.00030,6.61854


In [154]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X)

X[X.columns] =   scaler.transform(X)

In [155]:
X

,MedianF_Alpha_F4,MeanF_Alpha_F4,MedianF_Alpha_C4,MeanP_Alpha_O2,MeanF_Alpha_C4,MedianF_Delta_C4,MedianF_Delta_F4,MedianF_Delta_O2,MeanF_Alpha_O2,MedianF_Alpha_O2,MeanF_Beta_F4,SpectralEdge_Beta_F4,MeanF_Beta_C4,MeanP_Beta_F4,MeanF_Delta_C4
0,0.630879,0.703047,0.188717,-0.090035,0.578725,-0.199981,-0.201811,-0.187368,0.516306,0.616104,-0.106329,-0.080005,-0.090191,-0.079722,0.149598
1,0.969324,0.638237,0.362572,-0.090035,0.803059,-0.199981,-0.201811,-0.118640,0.447183,0.528886,-0.115262,-0.114617,-0.090191,-0.077219,0.113126
2,0.224744,0.638237,-0.054681,-0.086551,0.354390,0.006967,-0.132965,0.018821,0.308936,0.310837,-0.052727,-0.010780,-0.055935,-0.092232,0.040177
3,0.427814,0.216977,0.293029,-0.079583,0.304537,-0.199981,-0.201811,-0.153003,0.193731,0.223616,-0.365405,-0.426128,-0.338545,-0.069713,0.404916
4,0.630879,0.346595,0.188717,-0.095261,0.304537,-0.234473,-0.236236,-0.153003,0.262854,0.398055,-0.338604,-0.356904,-0.295725,-0.112249,0.076652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89091,0.224744,0.638237,0.119174,-0.090035,0.329464,-0.062015,-0.132965,-0.118640,0.355017,0.180006,-0.293936,-0.313638,-0.278597,-0.132266,0.003703
89092,0.766260,1.286333,0.397340,-0.053453,0.952615,-0.165491,-0.270661,-0.256099,0.815841,0.528886,0.242084,0.465139,0.226673,0.010355,0.113126
89093,0.698569,0.865071,0.432112,-0.049970,0.628577,-0.165491,-0.201811,-0.187368,0.516306,0.485276,0.054477,0.119016,0.029704,-0.032181,0.222547
89094,0.157059,0.346595,0.014862,-0.074357,0.130056,-0.027523,-0.236236,-0.221733,0.147648,0.136399,-0.204599,-0.253066,-0.312852,-0.122258,0.113126


#Spliting X and y to train and test sets

In [156]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)

In [157]:
y_train.value_counts()

1    40942
0    17130
2    13204
Name: Sleep_Stage, dtype: int64

#Oversampling using SMOTE

**<font color='red'>SNB:</font><font color='olive'> We used *SMOTE* in train data only, so that our test data can remain corrupted data free. The accurecy of the model will be counted only for actual data.</font>**

In [164]:
!pip install imbalanced-learn

In [159]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train,y_train = smote.fit_resample(X_train,y_train)

In [160]:
y_train.value_counts()

2    40942
1    40942
0    40942
Name: Sleep_Stage, dtype: int64

In [161]:
y_test.value_counts()

1    10399
0     4160
2     3261
Name: Sleep_Stage, dtype: int64

#RandomForest

In [162]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

CPU times: user 51.9 s, sys: 83.9 ms, total: 52 s
Wall time: 1min 13s


In [163]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[3364  782   14]
 [ 768 8820  811]
 [  26  802 2433]]
              precision    recall  f1-score   support

           0       0.81      0.81      0.81      4160
           1       0.85      0.85      0.85     10399
           2       0.75      0.75      0.75      3261

    accuracy                           0.82     17820
   macro avg       0.80      0.80      0.80     17820
weighted avg       0.82      0.82      0.82     17820

Accurecy:  0.8202581369248035


#AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

#Gradient_Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
grd = GradientBoostingClassifier()
grd.fit(X_train, y_train)
y_pred = grd.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))